In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input,Embedding,LSTM,Dense,GlobalMaxPooling1D,Flatten
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

In [18]:
%%writefile content.json
{"intents":
  [
     { 
      "tag":"greeting",
      "input":["hello","Hi there","nice to meet you","Hi this is the Organization", "Hi","Hey there", "Hey","hi there","Hello there"],
      "responses":[
              "Hi, Welcome to the World's Largest Organization",
              "Hey you, How may i help you ?",
              "hey" ,
              "Hi, What do you want?",
              "Hello my friend, How may I help you?",
              "Hi sir, How are you ?",
              "Hello , How can I help you ?",
              "Hey, what do you want ?",
              "Hi"
              ]
      },
   {
    "tag":"goodbye",
    "input":["Thank you","Thanks for the info", "Byee","Bye", "Okay Bye","goodbye","see you later","I'll catch you later", "Thank you and goodbye"],
    "responses":[
                 "okay,Bye",
                 "have a nice day",
                 "see you later",
                 "nice conversation, bye",
                 "okay, have a nice day",
                 "take care",
                 "Byee",
                 "okay take care",
                 "Bye, see you later"
              ]
   },

   {
     "tag":"Howami",
    "input":["How are you ?","I am fine, How are you ?", "are you fine ?","How are things going on ?","everthing going on ok ?", "How's everything going on ?","Hey, How you doin?"],
    "responses":[
                 "Yeah, I am fine",
                 "Everythins's going on well",
                 "Doing good, thanks for asking",
                 "Yeah Fine, nothing much",
                 "I am great",
                 "everything is going on great",
                 "I am doing good"
    ]
   },
   {
    "tag":"whoareyou",
    "input":["Who are you ?","What are you ?", "What is your name ?","Are you a chatbot ?","what can i call you ?","your name ?","are you a bot?"],
    "responses":[
                "I am oraganization's bot",
                "I am oot and you can ask me about our organzation",
                "My name is bot and i am a chatbot",
                "myself bot, how can i help you ?",
                "you can call me bot and i am here to help you",
                "I am bot and i am here to answer your questions",
                "going bot at your service"
    
    ]
   },
   {
    "tag":"whereareyou",
    "input":["Where are you from ?","which country are you from ?","Where do you live ?","Where are you ?","Where do you live in?","which place do you  belong?"],
    "responses":[
                 "I live in East blue",
                 "I am setting sail to the grand line",
                 "I live in East Blue,Pretty sure u would't have heard of it",
                 "East Blue, thats where i live"
    ]
       
   },
   {
    "tag":"join",
    "input":["How to get recruited?","How to join?","How can i get recruited ?","How to join this organistion?","What is the procedure to join this organization?"],
    "responses":[
                 "Your need to contact our nearest centre once you complete the registeration",
                 "Contact closest centre from your location",
                 "well ther are some pre-requisites and completing them,find nearest centre"
    ]

   },
   {
    "tag":"whereisthecentre",
    "input":["Where is organisation's centre located ?","where is the location ?","where is it located ?","Where can i find the centre ?"],
    "responses":[
                 "Thats upto you, you can use googlemaps",
                 "They are present in many places in the city you can choose the nearest one",
                 "There are lot of centre's in your city you can find them"
    ]
   },
   {
    "tag":"prerequisites",
    "input":["What are job requirements ?","what are the pre requisites?","what are the pre-requisites ?","tell me about the job requirements"]  ,
    "responses":[
                 "You can find it on our official site",
                 "Find it in the centre near you",
                 "You need proffesionals skills for this job"
                  
    ]

   }
  ]

}

Overwriting content.json


In [19]:
with open('content.json') as content:
  data1 = json.load(content)

In [21]:
tags =[]
inputs = []
responses = {}
for intent in data1['intents']:
  responses[intent['tag']] = intent['responses']
  for lines in intent['input']:
    inputs.append(lines)
    tags.append(intent['tag'])
      

In [22]:
data = pd.DataFrame({'inputs':inputs,
                     "tags":tags})

In [24]:
data.head()

,inputs,tags
0,hello,greeting
1,Hi there,greeting
2,nice to meet you,greeting
3,Hi this is the Organization,greeting
4,Hi,greeting


In [25]:
data = data.sample(frac=1)

In [27]:
import string
data['inputs'] = data['inputs'].apply(lambda wrd:[ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation ])
data['inputs'] = data['inputs'].apply(lambda wrd: ''.join(wrd))
data.head()

,inputs,tags
28,are you a chatbot,whoareyou
35,where are you,whereareyou
49,what are the prerequisites,prerequisites
36,where do you live in,whereareyou
30,your name,whoareyou


In [28]:
tokenizer = Tokenizer(num_words = 2000)
tokenizer.fit_on_texts(data['inputs'])
train = tokenizer.texts_to_sequences(data['inputs'])

from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train = pad_sequences(train)

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(data['tags'])

In [31]:
 input_shape = x_train.shape[1]
 print(input_shape)

8


In [34]:
vocabulary = len(tokenizer.word_index)
print('no. of unique words: ', vocabulary)
output_length = le.classes_.shape[0]
print('output_shape: ',output_length)

no. of unique words:  75
output_shape:  8


In [36]:
i = Input(shape =(input_shape,))
x = Embedding(vocabulary+1,10)(i)
x = LSTM(10,return_sequences= True)(x)
x = Flatten()(x)
x = Dense(output_length,activation= 'softmax')(x)
model = Model(i,x)

In [39]:
model.compile(loss='sparse_categorical_crossentropy',optimizer = 'adam',metrics=['accuracy'])

In [41]:
train = model.fit(x_train,y_train,epochs= 200)

Epoch 1/200
2/2 [==============================] - 0s 16ms/step - loss: 1.7550 - accuracy: 0.3922
Epoch 2/200
2/2 [==============================] - 0s 15ms/step - loss: 1.7391 - accuracy: 0.4118
Epoch 3/200
2/2 [==============================] - 0s 13ms/step - loss: 1.7225 - accuracy: 0.4118
Epoch 4/200
2/2 [==============================] - 0s 11ms/step - loss: 1.7055 - accuracy: 0.4118
Epoch 5/200
2/2 [==============================] - 0s 11ms/step - loss: 1.6879 - accuracy: 0.4118
Epoch 6/200
2/2 [==============================] - 0s 15ms/step - loss: 1.6692 - accuracy: 0.4118
Epoch 7/200
2/2 [==============================] - 0s 13ms/step - loss: 1.6502 - accuracy: 0.4706
Epoch 8/200
2/2 [==============================] - 0s 16ms/step - loss: 1.6310 - accuracy: 0.4706
Epoch 9/200
2/2 [==============================] - 0s 13ms/step - loss: 1.6105 - accuracy: 0.4706
Epoch 10/200
2/2 [==============================] - 0s 13ms/step - loss: 1.5895 - accuracy: 0.4706
Epoch 11/200
2/2 [=

In [48]:
import random

while True:
  texts_p = []
  prediction_input = input('You: ')

  prediction_input= [letters.lower() for letters in prediction_input if letters not in string.punctuation]
  prediction_input = ''.join(prediction_input)
  texts_p.append(prediction_input)

  prediction_input = tokenizer.texts_to_sequences(texts_p)
  prediction_input = np.array(prediction_input).reshape(-1)
  prediction_input = pad_sequences([prediction_input],input_shape)

  output = model.predict(prediction_input)
  output = output.argmax()

  response_tag = le.inverse_transform([output])[0]
  print('BOT : ',random.choice(responses[response_tag])) 
  if response_tag == 'goodbye':
    break

You: Hie
BOT :  Hello my friend, How may I help you?
You: What is you name
BOT :  I am oot and you can ask me about our organzation
You: where do you live
BOT :  I live in East Blue,Pretty sure u would't have heard of it
You: canyou tell about your organization
BOT :  I am oraganization's bot
You: job details ?
BOT :  hey
You: bye
BOT :  take care


In [45]:
import random